In [1]:
import pathlib
import sys
import time

sys.path.append("../featurization")
import cucim.skimage.measure
import cucim.skimage.segmentation
import cupy
import cupyx.scipy.ndimage
import numpy as np
import pandas as pd
import scipy
import skimage
from data_writer import organize_featurization_data
from intensity import measure_3D_intensity_gpu
from loading_classes import ImageSetLoader, ObjectLoader

try:
    cfg = get_ipython().config
    in_notebook = True
except NameError:
    in_notebook = False
if in_notebook:
    from tqdm.notebook import tqdm
else:
    from tqdm import tqdm

In [2]:
image_set_path = pathlib.Path("../../data/NF0014/cellprofiler/C4-2/")

In [3]:
channel_mapping = {
    "DNA": "405",
    "AGP": "488",
    "ER": "555",
    "Mito": "640",
    "BF": "TRANS",
    "Nuclei": "nuclei_",
    "Cell": "cell_",
    "Cytoplasm": "cytoplasm_",
    "Organoid": "organoid_",
}

In [4]:
image_set_loader = ImageSetLoader(
    image_set_path=image_set_path,
    spacing=(1, 0.1, 0.1),
    channel_mapping=channel_mapping,
)

In [5]:
start_time = time.time()

In [6]:
for compartment in tqdm(
    image_set_loader.compartments, desc="Processing compartments", position=0
):
    for channel in tqdm(
        image_set_loader.image_names,
        desc="Processing channels",
        leave=False,
        position=1,
    ):
        object_loader = ObjectLoader(
            image_set_loader.image_set_dict[channel],
            image_set_loader.image_set_dict[compartment],
            channel,
            compartment,
        )

        output_dict = measure_3D_intensity_gpu(object_loader)
        final_df = pd.DataFrame(output_dict)
        # prepend compartment and channel to column names
        final_df.columns = [
            f"{compartment}_{channel}_{col}" for col in final_df.columns
        ]
        final_df["image_set"] = image_set_loader.image_set_name

        output_file = pathlib.Path(
            f"../results/{image_set_loader.image_set_name}/Intensity_{compartment}_{channel}_features.parquet"
        )
        output_file.parent.mkdir(parents=True, exist_ok=True)
        final_df.to_parquet(output_file)

Processing compartments:   0%|          | 0/4 [00:00<?, ?it/s]

Processing channels:   0%|          | 0/5 [00:00<?, ?it/s]

/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/cupyx/scipy/ndimage/_measurements.py:397: PerformanceWarning: Using the slower implementation as cupyx.scipy.ndimage.sum supports int32, float16, float32, float64, uint32, uint64 as data typesfor the fast implmentation
  warnings.warn(
/home/lippincm/miniforge3/envs/GFF_featurization/lib/python3.12/site-packages/cupyx/scipy/ndimage/_measurements.py:487: PerformanceWarning: Using the slower implementation as cupyx.scipy.ndimage.mean supports int32, float16, float32, float64, uint32, uint64 as data types for the fast implmentation
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
print(f"Intensity time: {time.time() - start_time}")

In [ ]:
df = pd.DataFrame(output_dict)
df.head()